# Use GCP bucket to host images of cats

## install libs

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [ ]:
# Install Google Cloud SDK
!pip install -q google-cloud-storage
# !apt-get -q install google-cloud-sdk

Reading package lists...
Building dependency tree...
Reading state information...
E: Unable to locate package google-cloud-sdk


## Authent step

In [ ]:
# Authenticate
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Initialize the Google Cloud SDK
!gcloud init


Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
core:
  account: Louispaulet13@gmail.com

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for this 
configuration:
 [1] Louispaulet13@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [Louispaulet13@gmail.com].

Pick cloud project to use: 
 [1] gen-lang-client-0077006347
 [2] hatvp-backup
 [

## Simple text file upload test

In [ ]:
! echo "this is a test file" > test_file.txt

In [ ]:
from google.cloud import storage

def upload_to_bucket(blob_name, file_path, bucket_name):
    """ Upload data to a bucket"""
    # Explicitly use service account credentials by specifying the private key file.
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    print(f"File {file_path} uploaded to {blob_name}.")

# Example usage
file_path = "/content/test_file.txt"  # Replace with the path to your file
blob_name = "test_file.txt"  # The name of the file in GCP bucket

upload_to_bucket(blob_name, file_path, "pretty_cats")


File /content/test_file.txt uploaded to test_file.txt.


## Get the cat pictures in a local dir

In [ ]:
from datasets import load_dataset

cats_ds = load_dataset('the-french-artist/unique_9k_cats', split='train')
cats_ds

Generating train split:   0%|          | 0/9936 [00:00<?, ? examples/s]

Dataset({
    features: ['image', 'label'],
    num_rows: 9936
})

In [ ]:
cats_df = cats_ds.to_pandas()

In [ ]:
import os
import pandas as pd
from PIL import Image
import io
from tqdm.auto import tqdm

def save_images_from_dataframe(df, folder_name):
    # Create the directory if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Iterate over the rows in the dataframe
    for index, row in tqdm(df.iterrows()):
        # Get the binary data
        image_data = row['image']

        # Convert binary data to an image
        image = Image.open(io.BytesIO(image_data['bytes']))

        # Define the filename
        filename = os.path.join(folder_name, f'image_{index}.jpg')

        # Save the image
        image.save(filename)

# Example usage
# Assuming your dataframe is loaded into the variable cats_df
save_images_from_dataframe(cats_df, 'cat_export')


0it [00:00, ?it/s]

## Upload local cat pics to remote bucket

### load list of file paths

In [ ]:
from glob import glob

glob_path = glob('cat_export/*.jpg')
glob_path[:5]

['cat_export/image_1260.jpg',
 'cat_export/image_2434.jpg',
 'cat_export/image_8774.jpg',
 'cat_export/image_6774.jpg',
 'cat_export/image_2702.jpg']

### create list of filenames

In [ ]:
filename_list = []
for filepath in glob_path:
  filename_list.append(filepath.split('/')[-1])
filename_list[:5]

['image_1260.jpg',
 'image_2434.jpg',
 'image_8774.jpg',
 'image_6774.jpg',
 'image_2702.jpg']

In [ ]:
for filepath, filename in tqdm(zip(glob_path, filename_list)):
  upload_to_bucket(filename, filepath, "pretty_cats")

0it [00:00, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
File cat_export/image_4902.jpg uploaded to image_4902.jpg.
File cat_export/image_3663.jpg uploaded to image_3663.jpg.
File cat_export/image_3813.jpg uploaded to image_3813.jpg.
File cat_export/image_699.jpg uploaded to image_699.jpg.
File cat_export/image_416.jpg uploaded to image_416.jpg.
File cat_export/image_5994.jpg uploaded to image_5994.jpg.
File cat_export/image_2436.jpg uploaded to image_2436.jpg.
File cat_export/image_8512.jpg uploaded to image_8512.jpg.
File cat_export/image_5381.jpg uploaded to image_5381.jpg.
File cat_export/image_7402.jpg uploaded to image_7402.jpg.
File cat_export/image_5770.jpg uploaded to image_5770.jpg.
File cat_export/image_7289.jpg uploaded to image_7289.jpg.
File cat_export/image_3690.jpg uploaded to image_3690.jpg.
File cat_export/image_6048.jpg uploaded to image_6048.jpg.
File cat_export/image_5514.jpg uploaded to image_5514.jpg.
File cat_export/image_2971.jpg uploaded t